In [5]:
import tensorflow as tf
import numpy as np

trainToTestRatio = 0.8

dataset = np.load('faceFeatures_9216_normalized_labels_as_coord.npz')
samples = dataset['data'].shape[0]
sampleSize = dataset['data'].shape[1]
coordsCount = dataset['y'].shape[1]
splitIdx = int(samples*trainToTestRatio)
trainData = dataset['data'][:splitIdx]
trainY = dataset['y'][:splitIdx]
testData = dataset['data'][splitIdx:]
testY = dataset['y'][splitIdx:]

In [6]:
hiddenLayerSize = 100
outputSize = coordsCount

x = tf.placeholder(tf.float32, shape=[None, sampleSize])
y_ = tf.placeholder(tf.float32, shape=[None, outputSize])

# Hidden

W_hidden = tf.Variable(tf.truncated_normal((sampleSize, hiddenLayerSize), stddev=0.1))
b_hidden = tf.Variable(tf.constant(0.1, shape=(hiddenLayerSize,)))

hidden = tf.nn.relu(tf.matmul(x, W_hidden) + b_hidden)

# Output

W_output = tf.Variable(tf.truncated_normal((hiddenLayerSize, outputSize), stddev=0.1))
b_output = tf.Variable(tf.constant(0.1, shape=(outputSize,)))
y = tf.matmul(hidden, W_output) + b_output

# add overfitting control

In [13]:
#lossFunc = tf.nn.l2_loss(y - y_)
lossFunc = tf.sqrt(tf.reduce_mean((y - y_)**2))

train_step = tf.train.AdamOptimizer(1e-4).minimize(lossFunc)

In [ ]:
epohs = 5000


init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

trainLoss = []
testLoss = []

for i in range(epohs):
    evalY=y.eval(feed_dict={x:trainData},session=sess)
    evalY[~np.isnan(trainY)]=trainY[~np.isnan(trainY)]
    if i % 100 == 0:
        train = lossFunc.eval(feed_dict={x:trainData, y_:evalY}, session=sess)
        evalTestY=y.eval(feed_dict={x:testData},session=sess)
        evalTestY[~np.isnan(testY)]=testY[~np.isnan(testY)]
        test = lossFunc.eval(feed_dict={x:testData, y_:evalTestY}, session=sess)
        
        print('train:', train, 'test:', test)
    train_step.run(feed_dict={x:trainData, y_:evalY}, session=sess)

train: 2.17743 test: 1.15023
train: 0.294141 test: 0.182157
